*READ ME*

This notebook is used to collect youtube data via YouTube Data API. Please check the official document before moving on: https://developers.google.com/youtube/v3/getting-started

To use this notebook, one or multiple (recommended) API keys are needed. The daily quotas of one API key can be used to fetch complete data for about 5 artists.

The artist YouTube channel ID should be identified before the request. The channel ID can be found in the channel URL. For example, https://www.youtube.com/channel/UCuHzBCaKmtaLcRAOoazhCPA is the link for Beyoncé's channel, so the channel ID is UCuHzBCaKmtaLcRAOoazhCPA .

In [3]:
import pandas as pd
import numpy as np
import random
import json
import urllib
import dateutil.parser
from dateutil.tz import gettz
from dateutil import tz
from datetime import datetime
import time

Add or remove API keys

In [4]:
api_list = [    
# put your keys here
]

Identify the artist names and corresponding YouTube channel IDs, and make sure your dataframe is like following:

In [6]:
youtubeid = pd.read_csv("youtubeid.csv",header = None)
youtubeid = pd.DataFrame({'0':['SZA','The Chainsmokers'],'1':['UCO5IQ70V7l-XpHW40HwaGsw','UCq3Ci-h945sbEYXpVlw7rJg']})
# youtubeid = youtubeid.iloc[110:115,:]
youtubeid

,0,1
0,SZA,UCO5IQ70V7l-XpHW40HwaGsw
1,The Chainsmokers,UCq3Ci-h945sbEYXpVlw7rJg


In [210]:
def get_all_video_in_channel(artist,channel_id, api_list):
    api_key = api_list[-1]
    a_num = 0
    video_time = []
    video_name = []
    play_list = []
    video_links = []
    col_1 = []
    col_2 = []
    col_3 = []  
    col_4 = []
    col_5 = []
    col_6 = []
    col_7 = []
    col_8 = []
    col_9 = []
    col_10 =[]

    
    base_video_url = 'https://www.youtube.com/watch?v='
    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'
    base_playlist_url = 'https://www.googleapis.com/youtube/v3/playlists?'
    first_url = base_playlist_url+'key={}&maxResults=50&part=contentDetails&channelId={}'.format(api_key, channel_id)
    
    base_channel_url = 'https://www.googleapis.com/youtube/v3/channels?'
    first_channel_url = base_channel_url+'key={}&maxResults=50&part=statistics,snippet&id={}'.format(api_key, channel_id)
    channel_url = first_channel_url
    
    channel_inp = urllib.request.urlopen(channel_url)
    channel_resp = json.load(channel_inp)
    channel_view_count = channel_resp['items'][0]['statistics']['viewCount']
    channel_subscriber_count = channel_resp['items'][0]['statistics']['subscriberCount']

    url = first_url
    while True:
        inp = urllib.request.urlopen(url)
        resp = json.load(inp)
        
        for i in resp['items']:
            if i['kind'] == "youtube#playlist":
                play_list.append(i['id'])
    
        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
            
    for i in play_list:
        first_url = 'https://www.googleapis.com/youtube/v3/playlistItems?key={}&maxResults=50&part=contentDetails,snippet&playlistId={}'.format(api_key,i)
        url = first_url
        while True:
            inp = urllib.request.urlopen(url)
            resp = json.load(inp)

            for i in resp['items']:
                if i['kind'] == "youtube#playlistItem":
                    if i['snippet']["resourceId"]['kind']=='youtube#video':
                        col_1.append(artist)
                        col_2.append(i['snippet']['title'])
                        col_3.append(i['snippet']['publishedAt'])
                        col_9.append(channel_view_count)
                        col_10.append(channel_subscriber_count)
                        video_id = i['snippet']["resourceId"]['videoId']
                        video_url = 'https://www.googleapis.com/youtube/v3/videos?part=statistics&id='+video_id+'&key='+api_list[a_num]
                        video_stats = ''
                        try:
                            video_stats = urllib.request.urlopen(video_url)
                        except:
                            print('Key number '+str(a_num))
                            original = a_num
                            a_num = random.randint(0, len(api_list)-1)
                            while a_num==original:
                                a_num = random.randint(0, len(api_list)-1)
                            print('Switching key to ' +str(a_num))
                            video_url = 'https://www.googleapis.com/youtube/v3/videos?part=statistics&id='+video_id+'&key='+api_list[a_num]
                            video_stats = urllib.request.urlopen(video_url)
                        vstats = json.load(video_stats)
                        if vstats["pageInfo"]["totalResults"]>0:
                            try:
                                col_4.append(vstats["items"][0]['statistics']["viewCount"])
                            except:
                                col_4.append(0)
                            try:
                                col_5.append(vstats["items"][0]['statistics']["likeCount"])
                            except:
                                col_5.append(0)
                            try:
                                col_6.append(vstats["items"][0]['statistics']["dislikeCount"])
                            except:
                                col_6.append(0)
                            try:
                                col_7.append(vstats["items"][0]['statistics']["favoriteCount"])
                            except:
                                col_7.append(0)
                            try:
                                col_8.append(vstats["items"][0]['statistics']["commentCount"])
                            except:
                                col_8.append(0)
                        else:
                            col_4.append(0)
                            col_5.append(0)
                            col_6.append(0)
                            col_7.append(0)
                            col_8.append(0)
                        if a_num==len(api_list)-1:
                            return [col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8]
            try:
                next_page_token = resp['nextPageToken']
                url = first_url + '&pageToken={}'.format(next_page_token)
            except:
                break
    return [col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10]

def get_results(youtubeid,api_key,all_data=[]):
    for i in range(len(youtubeid)):
        print('Collecting: ', youtubeid.iloc[i,0])
        result = get_all_video_in_channel(youtubeid.iloc[i,0],youtubeid.iloc[i,1],api_key)
        rdf = pd.DataFrame(np.array(result).T)
        all_data.append(rdf)
    return all_data

def get_hour(d):
    return d.hour

def time_of_day_creation(x):
    '''
    Takes in an integer and returns one of four time slots:
    ['12am-6am', '6am-12pm', '12pm-6pm', '6pm-12am']
    '''
    
    slots_out = ['12am-6am', '6am-12pm', '12pm-6pm', '6pm-12am']
    slot0 = np.arange(0,6)
    slot1 = np.arange(6,12)
    slot2 = np.arange(12,18)
    slot3 = np.arange(18,24)

    if x in slot0:
        return slots_out[0]
    elif x in slot1:
        return slots_out[1]
    elif x in slot2:
        return slots_out[2]
    elif x in slot3:
        return slots_out[3]
    else:
        return 'NaN'
    
def get_length(d):
    return len(d)

def calculateRollupPosts(social_media_df,social_media_type, social_media_handle_field):
    time_range = pd.date_range(start='1/1/2016', end='6/30/2019', freq='d').to_frame(index=False)
    time_range.rename(columns={0: 'Time Period'}, inplace=True)
    master_artists_list = pd.read_csv('master_artists_list.csv', usecols=[social_media_type])
    time_range['join_key'] = 0
    master_artists_list['join_key'] = 0
    artists_time_range = master_artists_list.merge(time_range, how='left', on = 'join_key')
    artists_time_range.drop('join_key',1, inplace=True)
    artists_time_range.rename(columns={0: 'Time Period'}, inplace=True)
    social_media_df['Created_Day'] = social_media_df.created.dt.floor('d')
    df_aggregate = social_media_df.groupby([social_media_handle_field, 'Created_Day'], as_index=False).agg({"created": "count"})
    df_aggregate.rename(columns={'created': 'Posts In Day'}, inplace=True)
    df_merged = artists_time_range.merge(df_aggregate, how='left', left_on=[social_media_type,'Time Period'], right_on=[social_media_handle_field,'Created_Day'])
    df_merged['Posts In Day'] = df_merged['Posts In Day'].fillna(0)
    df_merged['last_7_days'] = df_merged[['Posts In Day']].groupby(df_merged[social_media_type]).apply(lambda g: g.rolling(7, min_periods=1).sum())
    df_merged['last_30_days'] = df_merged[['Posts In Day']].groupby(df_merged[social_media_type]).apply(lambda g: g.rolling(30, min_periods=1).sum())
    df_merged.drop(columns=[social_media_handle_field, 'Created_Day','Posts In Day'], inplace=True)
    social_media_df = social_media_df.merge(df_merged, how='left', right_on=[social_media_type,'Time Period'], left_on=[social_media_handle_field,'Created_Day'])
    social_media_df.drop(columns=[social_media_type, 'Time Period','Created_Day'], inplace=True)
    
    return social_media_df

def convertTZ(d):
    from_zone = tz.gettz('UTC')
    d = d.replace(tzinfo=from_zone)
    to_zone = tz.gettz('America/Pittsburgh')
    return d.astimezone(to_zone)

def get_int(s):
    return int(s)

def datetime_from_utc_to_local(utc_datetime):
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

def get_youtube_data(youtubeid,api_list):
    collected_data = get_results(youtubeid,api_list)
    combined_data = pd.concat(collected_data, ignore_index=True)
    combined_data.columns = ['artist','video','time','viewCount','likeCount','dislikeCount','favoriteCount','commentCount','viewChannel','subscriberChannel']
    combined_data['time'] = pd.to_datetime(combined_data['time'])
    combined_data["time"] =  combined_data["time"].apply(convertTZ)
    combined_data["time"] =  combined_data["time"].apply(datetime_from_utc_to_local)
    combined_data = combined_data.sort_values(by=['artist','time'])
    combined_data['since_last_post'] = combined_data.groupby('artist')['time'].diff()
    combined_data["hour"] = combined_data["time"].apply(get_hour)
    combined_data['time_of_day']=combined_data['hour'].apply(time_of_day_creation)
    combined_data['within_week_release'] = 0
    combined_data['within_month_release'] = 0
    combined_data = pd.concat([combined_data ,pd.get_dummies(combined_data ['time_of_day'],prefix='tod')],axis=1)
    combined_data['post_last_week'] = 0
    combined_data['post_last_month'] = 0
    combined_data=combined_data.reset_index(drop=True)
    combined_data = combined_data.rename(columns={"artist": "artist_name", 'time':"created",
                      "video":"description",'engage':'social_engagement_score',
                      'tod_12am-6am':'created_12am-6am',
                     'tod_6am-12pm':'created_6am-12pm',
                     'tod_12pm-6pm':'created_12pm-6pm',
                     'tod_6pm-12am':'created_6pm-12am',
                     'post_last_week':'posts_last_7days',
                     'post_last_month':'posts_last_30days',
                     'since_last_post':'time_since_last_post'
                     })
    combined_data['crowdtangle_score']=0
    combined_data['type_photo']=0
    combined_data['type_video']=1
    combined_data['type_text']=0
    combined_data['type_link']=0
    combined_data['count_of_followers'] = 0
    combined_data['hashtag_count'] = 0
    combined_data['description_length']=combined_data['description'].apply(get_length)
    combined_data['viewCount']=combined_data['viewCount'].apply(get_int)
    combined_data['likeCount'] = combined_data['likeCount'].apply(get_int)
    combined_data['commentCount'] = combined_data['commentCount'].apply(get_int)
    combined_data['social_engagement_score'] = combined_data['viewCount']+combined_data['likeCount']+combined_data['commentCount']
    combined_data = calculateRollupPosts(combined_data,'youtube','artist_name')
    combined_data=combined_data.drop(columns = ['likeCount','commentCount','viewCount',
                                                'dislikeCount','favoriteCount',
                                                'posts_last_30days','viewChannel',
                                                'subscriberChannel','posts_last_7days','time_of_day'])
    return combined_data
    
    

In [211]:
collected_data = get_youtube_data(youtubeid,api_list)
collected_data.head().T

Collecting:  Avril Lavigne


,0,1,2,3,4
artist_name,Avril Lavigne,Avril Lavigne,Avril Lavigne,Avril Lavigne,Avril Lavigne
description,Avril Lavigne - Making of the Black Star Comme...,Avril Lavigne - Black Star Launch Party,Avril Lavigne New Black Star Commercial - Offi...,In Too Deep,Avril Lavigne Tour Webisode
created,2010-02-18 14:50:33,2010-02-18 14:50:33,2010-02-18 14:50:33,2010-02-18 14:53:52,2010-02-18 14:53:52
time_since_last_post,NaT,0 days 00:00:00,0 days 00:00:00,0 days 00:03:19,0 days 00:00:00
hour,14,14,14,14,14
within_week_release,0,0,0,0,0
within_month_release,0,0,0,0,0
created_12pm-6pm,1,1,1,1,1
created_6am-12pm,0,0,0,0,0
created_6pm-12am,0,0,0,0,0


In [186]:
collected_data.to_csv('collected_data.csv')